# Face Landmarks Detection with MediaPipe Tasks

This notebook shows you how to use MediaPipe Tasks Python API to detect face landmarks from images.

# Preparation

In [1]:
!wget -O face_landmarker_v2_with_blendshapes.task -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task

## Visualization utilities

In [2]:
#@markdown We implemented some functions to visualize the face landmark detection results. <br/> Run the following cell to activate the functions.
import mediapipe as mp # type: ignore
from mediapipe import solutions # type: ignore
from mediapipe.framework.formats import landmark_pb2 # type: ignore
import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore


def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

def plot_face_blendshapes_bar_graph(face_blendshapes):
  # Extract the face blendshapes category names and scores.
  face_blendshapes_names = [face_blendshapes_category.category_name for face_blendshapes_category in face_blendshapes]
  face_blendshapes_scores = [face_blendshapes_category.score for face_blendshapes_category in face_blendshapes]
  # The blendshapes are ordered in decreasing score value.
  face_blendshapes_ranks = range(len(face_blendshapes_names))

  fig, ax = plt.subplots(figsize=(12, 12))
  bar = ax.barh(face_blendshapes_ranks, face_blendshapes_scores, label=[str(x) for x in face_blendshapes_ranks])
  ax.set_yticks(face_blendshapes_ranks, face_blendshapes_names)
  ax.invert_yaxis()

  # Label each bar with values
  for score, patch in zip(face_blendshapes_scores, bar.patches):
    plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{score:.4f}", va="top")

  ax.set_xlabel('Score')
  ax.set_title("Face Blendshapes")
  plt.tight_layout()
  plt.show()

def print_eyes_score(face_blendshapes):
  face_blendshapes_names = np.array([face_blendshapes_category.category_name for face_blendshapes_category in face_blendshapes])
  face_blendshapes_scores = np.array([face_blendshapes_category.score for face_blendshapes_category in face_blendshapes])
  eye_blendshapes_names = face_blendshapes_names[9:22]
  eye_blendshapes_scores = face_blendshapes_scores[9:22]
  
    

## Download test image

Let's grab a test image that we'll use later. The image is from [Unsplash](https://unsplash.com/photos/mt2fyrdXxzk).

In [3]:
# !wget -q -O image.png https://storage.googleapis.com/mediapipe-assets/business-person.png

import cv2

img = cv2.imread("image.png")
cv2.imshow("test",img)
cv2.waitKey(0)

-1

## Running inference and visualizing the results

Here are the steps to run face landmark detection using MediaPipe.

Check out the [MediaPipe documentation](https://developers.google.com/mediapipe/solutions/vision/face_landmarker/python) to learn more about configuration options that this task supports.


In [4]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create an FaceLandmarker object.
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

# STEP 3: Load the input image.
image = mp.Image.create_from_file("image.png")

# STEP 4: Detect face landmarks from the input image.
detection_result = detector.detect(image)
# STEP 5: Process the detection result. In this case, visualize it.
annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
cv2.imshow("result", cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)

c:\Users\student\.conda\envs\mediapipe\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


-1

In [13]:
# plot_face_blendshapes_bar_graph(detection_result.face_blendshapes[0])
# print(len((detection_result.face_blendshapes)))

face_blendshapes_names = [face_blendshapes_category.category_name for face_blendshapes_category in detection_result.face_blendshapes[0]]
face_blendshapes_scores = [face_blendshapes_category.score for face_blendshapes_category in detection_result.face_blendshapes[0]]
# face_blendshapes_names = np.array(face_blendshapes_names)
# face_blendshapes_scores = np.array(face_blendshapes_scores)
upward_score = face_blendshapes_scores[17] + face_blendshapes_scores[18]
downward_score = face_blendshapes_scores[11] + face_blendshapes_scores[12]
leftward_score = face_blendshapes_scores[14] + face_blendshapes_scores[15]
rightward_score = face_blendshapes_scores[13] + face_blendshapes_scores[16]
dir_score = np.array([["upward",upward_score], []])

eyes_blendshapes_names = face_blendshapes_names[9:23]
eyes_blendshapes_scores = face_blendshapes_scores[9:23]

print(face_blendshapes_names[9:23])


['eyeBlinkLeft', 'eyeBlinkRight', 'eyeLookDownLeft', 'eyeLookDownRight', 'eyeLookInLeft', 'eyeLookInRight', 'eyeLookOutLeft', 'eyeLookOutRight', 'eyeLookUpLeft', 'eyeLookUpRight', 'eyeSquintLeft', 'eyeSquintRight', 'eyeWideLeft', 'eyeWideRight']


In [ ]:
# import cv2
# cap = cv2.VideoCapture(0)
# # print(ret_cam)
# while True:
#     _, frame = cap.read()
#     cam_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)   
#     detection_result_cam = detector.detect(cam_image)
#     annotated_image = draw_landmarks_on_image(cam_image.numpy_view(), detection_result_cam)
#     cv2.namedWindow('cam', cv2.WINDOW_AUTOSIZE)
#     cv2.imshow("cam", annotated_image)
#     print(detection_result_cam.face_blendshapes[0])
#     cv2.waitKey(1)
#     if cv2.waitKey(1) & 0xFF == ord('q'): 
#         cap.release()
#         cv2.destroyAllWindows()
#         break


We will also visualize the face blendshapes categories using a bar graph.

AttributeError: 'list' object has no attribute 'category_name'

And print the transformation matrix.

In [ ]:
print(detection_result.facial_transformation_matrixes)

[array([[ 9.92205501e-01, -6.04903474e-02,  1.08946197e-01,
         1.18711746e+00],
       [ 9.10740122e-02,  9.48733926e-01, -3.02671373e-01,
         2.23701134e+01],
       [-8.50522369e-02,  3.10234487e-01,  9.46847558e-01,
        -6.80049210e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])]
